In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os

In [2]:
os.path.join('data/train.csv')

'data/train.csv'

In [3]:
df = pd.read_csv('data/train.csv')

In [4]:
df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [5]:
df.iloc[6]['comment_text']

'COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK'

In [6]:
df[df.toxic==1].head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
6,0002bcb3da6cb337,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,1,1,0,1,0
12,0005c987bdfc9d4b,Hey... what is it..\n@ | talk .\nWhat is it......,1,0,0,0,0,0
16,0007e25b2121310b,"Bye! \n\nDon't look, come or think of comming ...",1,0,0,0,0,0
42,001810bf8c45bf5f,You are gay or antisemmitian? \n\nArchangel WH...,1,0,1,0,1,1
43,00190820581d90ce,"FUCK YOUR FILTHY MOTHER IN THE ASS, DRY!",1,0,1,0,1,0


In [7]:
df[df.columns[2:]].iloc[6]

toxic            1
severe_toxic     1
obscene          1
threat           0
insult           1
identity_hate    0
Name: 6, dtype: int64

In [8]:
df.shape

(159571, 8)

# 1. Preprocess

In [9]:
from tensorflow.keras.layers import TextVectorization

In [10]:
X = df.comment_text

In [11]:
y = df[df.columns[2:]].values

In [14]:
MAX_WORDS = 200000

In [16]:
vectorizer = TextVectorization(max_tokens = MAX_WORDS,output_sequence_length = 1800,
                              output_mode = 'int')

Metal device set to: Apple M1 Pro


In [17]:
vectorizer.adapt(X.values)

In [39]:
vectorizer.get_vocabulary() #-----> Returns the current vocabulary of the layer.


['',
 '[UNK]',
 'the',
 'to',
 'of',
 'and',
 'a',
 'you',
 'i',
 'is',
 'that',
 'in',
 'it',
 'for',
 'this',
 'not',
 'on',
 'be',
 'as',
 'have',
 'are',
 'your',
 'with',
 'if',
 'article',
 'was',
 'or',
 'but',
 'page',
 'my',
 'an',
 'from',
 'by',
 'do',
 'at',
 'about',
 'me',
 'so',
 'wikipedia',
 'can',
 'what',
 'there',
 'all',
 'has',
 'will',
 'talk',
 'please',
 'would',
 'its',
 'no',
 'one',
 'just',
 'like',
 'they',
 'he',
 'dont',
 'which',
 'any',
 'been',
 'should',
 'more',
 'we',
 'some',
 'other',
 'who',
 'see',
 'here',
 'also',
 'his',
 'think',
 'im',
 'because',
 'know',
 'how',
 'am',
 'people',
 'why',
 'edit',
 'articles',
 'only',
 'out',
 'up',
 'when',
 'were',
 'use',
 'then',
 'may',
 'time',
 'did',
 'them',
 'now',
 'being',
 'their',
 'than',
 'thanks',
 'even',
 'get',
 'make',
 'good',
 'had',
 'very',
 'information',
 'does',
 'could',
 'well',
 'want',
 'such',
 'sources',
 'way',
 'name',
 'these',
 'deletion',
 'pages',
 'first',
 'help'

In [40]:
vectorizer_text = vectorizer(X.values)

In [41]:
len(X)

159571

In [43]:
vectorizer_text #--> 1800 represendts output sequence length

<tf.Tensor: shape=(159571, 1800), dtype=int64, numpy=
array([[  645,    76,     2, ...,     0,     0,     0],
       [    1,    54,  2489, ...,     0,     0,     0],
       [  425,   441,    70, ...,     0,     0,     0],
       ...,
       [32445,  7392,   383, ...,     0,     0,     0],
       [    5,    12,   534, ...,     0,     0,     0],
       [    5,     8,   130, ...,     0,     0,     0]])>

# Remember MCSHBAP
* M ----> MAP
* C ----> CACHE
* SH----> SHUFFLE
* BA----> BATCH
* P ----> PREFETCH

In [53]:
dataset = tf.data.Dataset.from_tensor_slices((vectorizer_text,y))
dataset = dataset.cache()
dataset = dataset.shuffle(160000)
dataset = dataset.batch(16)
dataset = dataset.prefetch(8) # ---->It prevents the bottlenecks

In [54]:
dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 1800), dtype=tf.int64, name=None), TensorSpec(shape=(None, 6), dtype=tf.int64, name=None))>

In [56]:
batch_X,batch_y = dataset.as_numpy_iterator().next()

In [58]:
batch_X,batch_y.shape

(array([[5495, 1220,   73, ...,    0,    0,    0],
        [   5,  390,   42, ...,    0,    0,    0],
        [1885,  468,  179, ...,    0,    0,    0],
        ...,
        [ 901, 1048,  880, ...,    0,    0,    0],
        [   8,   19,  569, ...,    0,    0,    0],
        [ 165,    6, 1024, ...,    0,    0,    0]]),
 (16, 6))

In [63]:
train = dataset.take(int(len(dataset)*.7))
val = dataset.skip((int(len(dataset)*.7))).take(int(len(dataset)* .2))
test = dataset.skip((int(len(dataset)*.9))).take(int(len(dataset)*.1))

In [65]:
len(test)

997

In [66]:
train.as_numpy_iterator().next()

(array([[  323,    10,    14, ...,     0,     0,     0],
        [ 6613,    16,  1484, ...,     0,     0,     0],
        [    2, 78966,    11, ...,     0,     0,     0],
        ...,
        [  485,  2359,    49, ...,     0,     0,     0],
        [  669,  4541,   244, ...,     0,     0,     0],
        [    8,   954,     3, ...,     0,     0,     0]]),
 array([[0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0]]))

# 2. Creating Sequential Model

In [67]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,LSTM,Bidirectional,Embedding,Dropout

In [70]:
model = Sequential()

model.add(Embedding(MAX_WORDS+1,32))

model.add(Bidirectional(LSTM(32,activation ='tanh')))

model.add(Dense(128,activation='relu'))

model.add(Dense(256,activation='relu'))

model.add(Dense(128,activation='relu'))

model.add(Dense(6,activation='sigmoid'))


In [71]:
model.compile(optimizer='Adam',loss='BinaryCrossentropy')

In [72]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, None, 32)          6400032   
                                                                 
 bidirectional_1 (Bidirecti  (None, 64)                16640     
 onal)                                                           
                                                                 
 dense (Dense)               (None, 128)               8320      
                                                                 
 dense_1 (Dense)             (None, 256)               33024     
                                                                 
 dense_2 (Dense)             (None, 128)               32896     
                                                                 
 dense_3 (Dense)             (None, 6)                 774       
                                                      

In [73]:
from tensorflow.keras.callbacks import TensorBoard
tensorboard = TensorBoard(log_dir="logs")

In [74]:
history = model.fit(train,epochs=1,validation_data=val,callbacks=tensorboard)

6981/6981 [==============================] - 1729s 247ms/step - loss: 0.0616 - val_loss: 0.0446


In [75]:
model.save("results/model_comment_toxicity.h5")

/Users/nanda/miniforge3/envs/env_tensor/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [76]:
! tensorboard --logdir logs/ --port 8000

Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.13.0 at http://localhost:8000/ (Press CTRL+C to quit)
^C


In [80]:
history.history

{'loss': [0.061625346541404724], 'val_loss': [0.044643692672252655]}

# 3. Make Predictions

In [92]:
input_text

<tf.Tensor: shape=(1800,), dtype=int64, numpy=array([   7, 7158,  397, ...,    0,    0,    0])>

In [93]:
np.expand_dims(input_text,0)

array([[   7, 7158,  397, ...,    0,    0,    0]])

In [94]:
model.predict(np.expand_dims(input_text,0))

1/1 [==============================] - 0s 67ms/step


array([[0.99167633, 0.19009407, 0.9201744 , 0.0237474 , 0.8038727 ,
        0.09857953]], dtype=float32)

In [95]:
model.predict(np.array([input_text]))

1/1 [==============================] - 0s 83ms/step


array([[0.99167633, 0.19009407, 0.9201744 , 0.0237474 , 0.8038727 ,
        0.09857953]], dtype=float32)

In [90]:
input_text = vectorizer('You freaking suck!')

In [96]:
df.columns[2:]

Index(['toxic', 'severe_toxic', 'obscene', 'threat', 'insult',
       'identity_hate'],
      dtype='object')

In [110]:
batch = test.as_numpy_iterator().next()

In [119]:
batch_X,batch_y = test.as_numpy_iterator().next()

In [120]:
batch_y

array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [1, 1, 1, 1, 1, 1],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [1, 1, 1, 0, 1, 0],
       [0, 0, 0, 0, 0, 0]])

In [123]:
(model.predict(batch_X) >0.35).astype(int)

1/1 [==============================] - 0s 74ms/step


array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [1, 1, 1, 0, 1, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [1, 1, 1, 0, 1, 0],
       [0, 0, 0, 0, 0, 0]])

# 4. Evaluate Model


In [126]:
from tensorflow.keras.metrics import Precision,Recall,CategoricalAccuracy

In [127]:
pre = Precision()
rec = Recall()
acc = BinaryAccuracy()

In [128]:
for batch in test.as_numpy_iterator():
    
    #unpack the batch 
    X_true,y_true = batch
    yhat = model.predict(X_true)
    
    #flatten the values
    
    y_true = y_true.flatten()
    yhat = yhat.flatten()
    
    pre.update_state(y_true,yhat)
    
    rec.update_state(y_true,yhat)
    
    acc.update_state(y_true,yhat)

1/1 [==============================] - 0s 60ms/step


In [129]:
print(f'Precison is : {pre.result().numpy()}, Accuracy is : {acc.result().numpy()}, Recall is {rec.result().numpy()}')

Precison is : 0.8311081528663635, Accuracy is : 0.9831891655921936, Recall is 0.6930142045021057


# 5. Test and Gradio App

In [133]:
! pip install gradio jinja2

  Using cached gradio-3.35.2-py3-none-any.whl (19.7 MB)
  Using cached aiohttp-3.8.4-cp38-cp38-macosx_11_0_arm64.whl (337 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.5/471.5 kB 483.5 kB/s eta 0:00:00 kB/s eta 0:00:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 670.6 kB/s eta 0:00:0031m1.9 MB/s eta 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.4/288.4 kB 545.0 kB/s eta 0:00:001m555.0 kB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 770.9 kB/s eta 0:00:00MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 520.1 kB/s eta 0:00:001m487.1 kB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.5/87.5 kB 726.0 kB/s eta 0:00:00 kB/s eta 0:00:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 1.1 MB/s eta 0:00:00 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 859.1 kB/s eta 0:00:00m eta 0:00:010:00:01
  Using cached pydub-0.

In [134]:
import gradio as gr

In [135]:
model = tf.keras.models.load_model('results/model_comment_toxicity.h5')

In [159]:
input_str = vectorizer('Hey idiot asshole motherfucker, i am warning you i willl kill you, ill hurt you .')

In [160]:
res = model.predict(np.expand_dims(input_str,0))

1/1 [==============================] - 0s 83ms/step


In [164]:
df.columns[2:]

Index(['toxic', 'severe_toxic', 'obscene', 'threat', 'insult',
       'identity_hate'],
      dtype='object')

In [165]:
res>0.35

array([[ True,  True,  True, False,  True, False]])

In [183]:
def score_comment(comment):
    vectorizer_comment = vectorizer([comment])
    results = model.predict(vectorizer_comment)
    text =''
    for idx,col in enumerate(df.columns[2:]):
        text+= '{}: {}\n'.format(col,results[0][idx]>0.2)
    return text

In [184]:
interface = gr.Interface(fn = score_comment,
                        inputs=gr.inputs.Textbox(lines=2,placeholder='Comment to score'),
                        outputs='text')

/Users/nanda/miniforge3/envs/env_tensor/lib/python3.8/site-packages/gradio/inputs.py:27: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
/Users/nanda/miniforge3/envs/env_tensor/lib/python3.8/site-packages/gradio/inputs.py:30: UserWarning: `optional` parameter is deprecated, and it has no effect
  super().__init__(
/Users/nanda/miniforge3/envs/env_tensor/lib/python3.8/site-packages/gradio/inputs.py:30: UserWarning: `numeric` parameter is deprecated, and it has no effect
  super().__init__(


In [186]:
interface.launch(share=True)

Rerunning server... use `close()` to stop if you need to change `launch()` parameters.
----
Running on local URL:  http://127.0.0.1:7862

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app. 

Also please ensure that your antivirus or firewall is not blocking the binary file located at: /Users/nanda/miniforge3/envs/env_tensor/lib/python3.8/site-packages/gradio/frpc_darwin_arm64_v0.2


1/1 [==============================] - 0s 128ms/step


In [192]:
# Filter the DataFrame to get rows where 'threat' is true
threat_data = df[df['threat'] == True]

# Extract the 'comment_text' column from the filtered data
threat_comments = threat_data['comment_text'].loc[159029]

# Print the extracted comments
print(threat_comments)

Death to Musulmans!
1/1 [==============================] - 0s 67ms/step
